In [1]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, AveragePooling2D,
                                   GlobalAveragePooling2D, BatchNormalization,
                                   Activation, Add, Concatenate, Dense, Dropout,
                                   Multiply, Reshape, Permute)
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, models

2025-07-11 02:47:12.790301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752202033.163258      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752202033.262269      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:

import tensorflow as tf

dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(64, 64),
    batch_size=16,
)

dataset_val = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',
    validation_split=0.2,
    subset="validation",
    
    seed=123,
    image_size=(64,64),
    batch_size=16,
)



Found 70295 files belonging to 38 classes.
Using 56236 files for training.
Found 17572 files belonging to 38 classes.
Using 3514 files for validation.


In [14]:

normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = dataset_train.map(lambda x, y: (normalization_layer(x), y))
val_ds = dataset_val.map(lambda x, y: (normalization_layer(x), y))

In [15]:


def simple_attention(x, reduction_ratio=8):
    
    channel = x.shape[-1]

    # Global Average Pooling
    gap = GlobalAveragePooling2D()(x)
    gap = layers.Reshape((1, 1, channel))(gap)

    attention = layers.Dense(channel // reduction_ratio, activation='relu')(gap)
    attention = layers.Dense(channel, activation='sigmoid')(attention)

    return Multiply()([x, attention])

In [17]:
list=[]
def residual_block(x, filters, stride=1):
    
    shortcut = x

    x = Conv2D(filters, 3, strides=stride, padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = Conv2D(filters, 3, padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)

    # Adjust shortcut if needed
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, 1, strides=stride, use_bias=False)(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x
def residual_block_group(x, filters, n_blocks, stride=1):
    x = residual_block(x, filters, stride=stride)
    for _ in range(1, n_blocks):
        x = residual_block(x, filters, stride=1)
    return x


def dense_block(x, num_layers, growth_rate):
    
    concat_features = [x]

    for _ in range(num_layers):
        x = Concatenate()(concat_features)
        out = BatchNormalization()(x)
        out = Activation('relu')(out)
        out = Conv2D(4 * growth_rate, (1, 1), padding='same', use_bias=False)(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
        out = Conv2D(growth_rate, (3, 3), padding='same', use_bias=False)(out)
        concat_features.append(out)

    x = Concatenate()(concat_features)
    return x

def transition_layer(x):
    
    x = BatchNormalization()(x)
    x = Conv2D(x.shape[-1] // 2, (1, 1), padding='same', use_bias=False)(x)
    x = AveragePooling2D(pool_size=(2, 2), strides=2)(x)
    return x

def gfa_residual_stream(input_tensor):

    global list
    x = Conv2D(64, 3, padding='same', use_bias=False)(input_tensor)

    x = BatchNormalization()(x)
    x = layers.ReLU()(x)

    


    x = residual_block_group(x, filters=128, n_blocks=2, stride=1)
    list.append(x)
    x = residual_block_group(x, filters=256, n_blocks=2, stride=2)
    list.append(x)
    x = residual_block_group(x, filters=512, n_blocks=2, stride=2)
    list.append(x)



    x = simple_attention(x)


    return GlobalAveragePooling2D()(x)
def gfa_parallel(list):
  pooled_list = []
  for i in list:
    i = simple_attention(i)
    pooled_list.append(GlobalAveragePooling2D()(i))
  list = Concatenate()(pooled_list)
  return list





def sf_dense_stream(input_tensor, growth_rate=16):
    
    x = Conv2D(64, (3, 3), padding='same', use_bias=False)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)


    x = dense_block(x, num_layers=4, growth_rate=growth_rate)
    x = transition_layer(x)

    x = dense_block(x, num_layers=4, growth_rate=growth_rate)
    x = transition_layer(x)

    x = dense_block(x, num_layers=8, growth_rate=growth_rate)

    sf_pool = GlobalAveragePooling2D()(x)
    return sf_pool







def build_derefnet(input_shape=(64, 64, 3), num_classes=38, growth_rate=16):
    
    from tensorflow.keras import Input
    inputs = Input(shape=input_shape)


    
    gfa_out = gfa_residual_stream(inputs)  
    sf_out = sf_dense_stream(inputs, growth_rate)
    gfa_parallel_out=gfa_parallel(list)


    fused = Concatenate()([gfa_out, sf_out, gfa_parallel_out])
    fused = Dropout(0.5)(fused)

    
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(fused)

    return Model(inputs, outputs, name='DeReFNet_PlantVillage')

In [18]:

print(tf.config.list_logical_devices())



[LogicalDevice(name='/device:CPU:0', device_type='CPU'), LogicalDevice(name='/device:GPU:0', device_type='GPU'), LogicalDevice(name='/device:GPU:1', device_type='GPU')]


In [19]:


model = build_derefnet(input_shape=(64,64,3), num_classes=38, growth_rate=16)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [20]:


model.fit(train_ds, validation_data=val_ds, epochs=30)


Epoch 1/30
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 569s 144ms/step - accuracy: 0.5153 - loss: 1.7005 - val_accuracy: 0.3452 - val_loss: 4.0254
Epoch 2/30
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 485s 138ms/step - accuracy: 0.8267 - loss: 0.5445 - val_accuracy: 0.7055 - val_loss: 1.0454
Epoch 3/30
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 482s 137ms/step - accuracy: 0.8869 - loss: 0.3561 - val_accuracy: 0.7223 - val_loss: 1.1577
Epoch 4/30
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 481s 137ms/step - accuracy: 0.9157 - loss: 0.2684 - val_accuracy: 0.8244 - val_loss: 0.6193
Epoch 5/30
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 482s 137ms/step - accuracy: 0.9307 - loss: 0.2149 - val_accuracy: 0.7584 - val_loss: 0.9343
Epoch 6/30
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 485s 138ms/step - accuracy: 0.9505 - loss: 0.1483 - val_accuracy: 0.8976 - val_loss: 0.3802
Epoch 8/30
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 480s 136ms/step - accuracy: 0.9574 - loss: 0.1333 - val_accuracy: 0.9419 - val_loss: 0.1715
Epoch 9/30
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 478s 136ms/step - ac

In [21]:
model.fit(train_ds, validation_data=val_ds, epochs=1)

3515/3515 ━━━━━━━━━━━━━━━━━━━━ 483s 137ms/step - accuracy: 0.9895 - loss: 0.0313 - val_accuracy: 0.9835 - val_loss: 0.0584


In [22]:
model.fit(train_ds, validation_data=val_ds, epochs=19)

Epoch 1/19
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 485s 138ms/step - accuracy: 0.9888 - loss: 0.0336 - val_accuracy: 0.9898 - val_loss: 0.0312
Epoch 2/19
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 483s 137ms/step - accuracy: 0.9900 - loss: 0.0293 - val_accuracy: 0.9846 - val_loss: 0.0400
Epoch 3/19
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 483s 138ms/step - accuracy: 0.9890 - loss: 0.0331 - val_accuracy: 0.9906 - val_loss: 0.0318
Epoch 4/19
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 482s 137ms/step - accuracy: 0.9898 - loss: 0.0320 - val_accuracy: 0.9277 - val_loss: 0.3591
Epoch 5/19
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 484s 138ms/step - accuracy: 0.9914 - loss: 0.0265 - val_accuracy: 0.9892 - val_loss: 0.0403
Epoch 6/19
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 483s 137ms/step - accuracy: 0.9899 - loss: 0.0309 - val_accuracy: 0.9237 - val_loss: 0.4323
Epoch 7/19
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 483s 137ms/step - accuracy: 0.9914 - loss: 0.0284 - val_accuracy: 0.9898 - val_loss: 0.0411
Epoch 8/19
3515/3515 ━━━━━━━━━━━━━━━━━━━━ 483s 137ms/step - ac

In [23]:
model.summary()

Model: "DeReFNet_PlantVillage"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_118 (Conv2D) │ (None, 64, 64,    │      1,728 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_118[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_66       │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ activation_66[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_42      │ (None, 32, 32,    │          0 │ max_pooling2d_2[… │
│ (Concatenate)       │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ concatenate_42[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_67       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_119 (Conv2D) │ (None, 32, 32,    │      4,096 │ activation_67[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_119[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_68       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_120 (Conv2D) │ (None, 32, 32,    │      9,216 │ activation_68[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_43      │ (None, 32, 32,    │          0 │ max_pooling2d_2[… │
│ (Concatenate)       │ 80)               │            │ conv2d_120[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        320 │ concatenate_43[0… │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_69       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_121 (Conv2D) │ (None, 32, 32,    │      5,120 │ activation_69[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_121[0][0]

 Total params: 34,553,860 (131.81 MB)

 Trainable params: 11,512,918 (43.92 MB)

 Non-trainable params: 15,104 (59.00 KB)

 Optimizer params: 23,025,838 (87.84 MB)